## 双延迟-确定性策略梯度（Twin Delayed Deterministic policy gradient，TD3）算法

&emsp;&emsp;<font size=4>
    在DDPG算法基础上，TD3算法的主要目的在于解决AC框架中，由函数逼近引入的偏差和方差问题。一方面，由于方差会引起过高估计，为解决过高估计问题，TD3将截断式双Q学习（clipped Double Q-Learning）应用于AC框架；另一方面，高方差会引起误差累积，为解决误差累积问题，TD3分别采用延迟策略更新和添加噪声平滑目标策略两种技巧。
</font><br>

### 过高估计问题解决方案

&emsp;&emsp;<font size=4>
    从策略梯度方法已知，基于PG的强化学习存在过高估计问题，但由于DDPG评论家的目标值不是取最优动作值函数的，所以不存在最大化操作。此时，将Double DQN思想直接用于DDPG的评论家，构造如下目标函数：
</font><br>
\begin{equation}
y=r+\gamma Q\left(s^{\prime}, \mu\left(s^{\prime}, \boldsymbol{\theta}\right), \boldsymbol{w}^{\prime}\right)  \label{bbb.11}\tag{bbb.11}
\end{equation}
&emsp;&emsp;<font size=4>
    实际上，这样的处理效果并不好，这是因为在连续动作空间中，策略变化缓慢，行动者更新较为平缓，使得预测$Q$值与目标$Q$值相差不大，无法避免过高估计问题。  
</font><br>
&emsp;&emsp;<font size=4>
    考虑将Double Q-Learning思想应用于DDPG，采用两个独立的评论家$Q_{w_{1}}$、$Q_{w_{2}}$和两个独立的行动者$\mu_{\theta_{1}}$、$\mu_{\theta_{2}}$，以50%的概率利用$Q_{1}$产生动作，然后更新$Q_{2}$估计值，而另外50%的概率正好相反。构建更新所需的两个目标值分别为：
</font><br> 
\begin{equation}
\left\{\begin{array}{l}
y_{1}=r+\gamma Q\left(s^{\prime}, \mu\left(s^{\prime}, \boldsymbol{\theta}_{1}\right), \boldsymbol{w}_{2}^{\prime}\right) \\
y_{2}=r+\gamma Q\left(s^{\prime}, \mu\left(s^{\prime}, \boldsymbol{\theta}_{2}\right), \boldsymbol{w}_{1}^{\prime}\right)
\end{array}\right.  \label{bbb.12}\tag{bbb.12}
\end{equation}
&emsp;&emsp;<font size=4>
    但由于样本均来自于同一经验池，不能保证样本数据完全独立，所以两个行动者的样本具有一定相关性，在一定的情况下，甚至会加剧高估问题。针对此种情形，秉持“宁可低估，也不要高估”的想法，对Double Q-Learning进行修改，构建基于Clipped Double Q-learning方法的目标值：
</font><br> 
\begin{equation}
y=r+\gamma \min _{i=1,2} Q\left(s^{\prime}, \mu\left(s^{\prime}, \theta_{1}\right), w_{i}^{\prime}\right) \label{bbb.13}\tag{bbb.13}
\end{equation}
&emsp;&emsp;<font size=4>
    如式（bbb.13）所示，目标值只使用了一个行动者网络$\mu_{\theta_{1}}$，取两个评论家网络$Q_{w_{1}}$和$Q_{w_{2}}$的最小值来作为值函数估计值。
</font><br>
&emsp;&emsp;<font size=4>
    在更新评论家网络$Q_{w_{1}}$和$Q_{w_{2}}$时，均采用式（bbb.13）目标值y，共用如下损失函数：
</font><br>
\begin{equation}
L\left(\boldsymbol{w}_{i}\right)=\mathbb{E}_{s, a, r, s^{\prime} \sim \mathcal{D}}\left[y-Q\left(s, a, \boldsymbol{w}_{i}\right)\right]^{2}    \label{bbb.14}\tag{bbb.14}    
\end{equation}
&emsp;&emsp;<font size=4>
    该算法相比于原算法的区别仅在于多了一个和原评论家$Q_{w_{1}}$同步更新的辅助评论家$Q_{w_{2}}$，在更新目标值y时取最小值。不过这一修改仍然会让人疑惑，$Q_{w_{1}}$和$Q_{w_{2}}$只有初始参数不同，后面的更新都一样，这样形成的两个类似的评论家能否有效消除TD误差带来的偏置估计。
</font><br>

### 累积误差问题解决方案

&emsp;&emsp;<font size=4>
    在函数逼近问题中，TD(0)算法的过高估计问题会进一步加剧，每次更新都会产生一定量的TD误差$\delta(s, a)$：
</font><br>
\begin{equation}
Q(s, a, \boldsymbol{w})=r+\gamma \mathbb{E}\left[Q\left(s^{\prime}, a^{\prime}, \boldsymbol{w}\right)\right]-\delta(s, a) \label{bbb.15}\tag{bbb.15}
\end{equation}
&emsp;&emsp;<font size=4>
    经过多次迭代更新后，误差会被累积：
</font><br>
\begin{equation}
\begin{aligned}
Q\left(S_{t}, A_{t}, \boldsymbol{w}\right) &=R_{t+1}+\gamma \mathbb{E}\left[Q\left(S_{t+1}, A_{t+1}, \boldsymbol{w}\right)\right]-\delta_{t+1} \\
&=R_{t+1}+\gamma \mathbb{E}\left[R_{t+2}+\gamma \mathbb{E}\left[Q\left(S_{t+2}, A_{t+2}, \boldsymbol{w}\right)\right]-\delta_{t+2}\right]-\delta_{t+1} \\
& \cdots \cdots \\
&=\mathbb{E}_{S_{i} \sim \rho^{\beta}, A_{i} \sim \mu}\left[\sum^{\mathrm{T}-1} \gamma^{i-t}\left(R_{i+1}-\delta_{i+1}\right)\right]
\end{aligned}  \label{bbb.16}\tag{bbb.16}
\end{equation}
&emsp;&emsp;<font size=4>
    由此可见，估计的方差与未来奖励、未来TD误差的方差成正比。当折扣因子$\gamma$较大时，每次更新都可以引起方差的快速提升，所以通常TD3设置较小的折扣系数$\gamma$。
</font><br>

### 延迟的策略更新

&emsp;&emsp;<font size=4>
    TD3目标网络的更新方式与DDPG相同，都采用软更新，尽管软更新比硬更新更有利于算法的稳定性，但AC算法依然会失败，其原因通常在于行动者和评论家的更新是相互作用的结果：评论家提供的值函数估计值不准确，就会使行动者将策略往错误方向改进；行动者产生了较差的策略，就会进一步加剧评论家误差累积问题，两者不断作用产生恶性循环。
</font><br>
&emsp;&emsp;<font size=4>
    为解决以上问题，TD3考虑对策略进行延时更新，减少行动者的更新频率，尽可能等待评论家训练收敛后再进行更新操作。延时更新操作可以有效减少累积误差，从而降低方差；同时，也能减少不必要的重复更新操作，一定程度上提升效率。在实际应用时，TD3采取的操作是每隔评论家更新$d$次后，再对行动者进行更新。
</font><br>

### 目标策略平滑操作

&emsp;&emsp;<font size=4>
    上节中通过延时更新策略来减小误差累积，接下来考虑误差本身。首先，误差的根源是值函数逼近所产生的偏差，在机器学习中，消除估计偏差的常用方法就是对参数更新进行正则化，同样的，这一思想也可以应用在强化学习中。
</font><br>
&emsp;&emsp;<font size=4>
    一个很自然的想法是，相似的动作应该拥有相似的价值，动作空间中目标动作周围的一小片区域的价值若能足够平滑，就可以有效减少误差的产生。TD3的具体做法是，为目标动作添加截断噪声：
</font><br>
\begin{equation}
\begin{array}{l}
\tilde{a} \leftarrow \mu\left(s^{\prime}, \boldsymbol{\theta}^{\prime}\right)+\varepsilon \\
\boldsymbol{\varepsilon} \sim \operatorname{clip}(N(0, \sigma),-c, c)
\end{array}   \label{bbb.17}\tag{bbb.17}
\end{equation}
&emsp;&emsp;<font size=4>
    该噪声处理也是一种正则化方式。通过这种平滑操作，可以增加算法的泛化能力，缓解过拟合问题，减少价值被过高估计的一些不良状态对策略学习的干扰。
</font><br>

## TD3算法

&emsp;&emsp;<font size=4>
    TD3算法，如算法bbb.2所示：
</font><br>

<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法bbb.2</b> TD3算法（Lillicrap al. 2016）</font><br>
<hr>
&emsp;&emsp;<font size=3.5>初始化：</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    1.&emsp;初始化预测价值网络$Q_{w_{1}}$和$Q_{w_{2}}$，网络参数分别为$\boldsymbol{w}_{1}$和$\boldsymbol{w}_{2}$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    2.&emsp;初始化目标价值网络$Q_{\boldsymbol{w}_{1}^{\prime}}$和$Q_{\boldsymbol{w}_{2}^{\prime}}$，网络参数分别为$\boldsymbol{w}_{1}^{\prime}$和$\boldsymbol{w}_{2}^{\prime}$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    3.&emsp;初始化预测策略网络$\mu_{\theta}$和目标策略网络$\mu_{\theta^{\prime}}$，网络参数分别为$\theta$和$\theta^{\prime}$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    4.&emsp;同步参数$w_{1}^{\prime} \leftarrow w_{1}$，$w_{2}^{\prime} \leftarrow w_{2}$，$\theta^{\prime} \leftarrow \theta$ 
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    5.&emsp;经验池$\mathcal{D}$的容量为$N$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    6.&emsp;总迭代次数$M$，折扣因子$\gamma$，$\tau=0.0001$，随机小批量采样样本数量$n$
</font><br>
<hr>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    7.&emsp;<b>for</b> $e$=1 <b>to</b> $M$ <b>do:</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    8.&emsp;&emsp;&emsp;初始化状态设置为$S_{0}$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    9.&emsp;&emsp;&emsp;<b>repeat</b>（情节中的每一时间步$t=0,1,2, \ldots$）：
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    10.&emsp;&emsp;&emsp;&emsp;&emsp;根据当前的预测策略网络和探索噪声来选择动作根据当前的预测策略网络和探索噪声来选择动作$A_{t}=\mu\left(S_{t}, \boldsymbol{\theta}\right)+\varepsilon_{t}$,
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    &emsp;&emsp;&emsp;&emsp;&emsp;其中$\varepsilon_{t} \sim \mathcal{N}_{t}(0, \sigma)$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    11.&emsp;&emsp;&emsp;&emsp;&emsp;执行动作$ A_{t}$，获得奖赏$R_{t+1}$和下一状态$S_{t+1}$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    12.&emsp;&emsp;&emsp;&emsp;&emsp;将经验转换$\left(S_{t}, A_{t}, R_{t+1}, S_{t+1}\right)$存储在经验池$\mathcal{D}$中
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    13.&emsp;&emsp;&emsp;&emsp;&emsp;从经验池$\mathcal{D}$中随机采样小批量的$n$个经验转移样本$\left(S_{i}, A_{i}, R_{i+1}, S_{i+1}\right)$，计算：
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    &emsp;&emsp;&emsp;&emsp;&emsp;（1）扰动后的动作$\tilde{a}_{i+1} \leftarrow \mu\left(S_{i+1}, \boldsymbol{\theta}^{\prime}\right)+\varepsilon_{i}$，其中$\varepsilon_{i} \sim \operatorname{clip}\left(\mathcal{N}_{t}(0, \tilde{\sigma}),-c, c\right)$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    &emsp;&emsp;&emsp;&emsp;&emsp;（2）更新目标$y_{i}=R_{i+1}+\gamma \min _{i=1,2} Q\left(S_{i+1}, \tilde{a}_{i+1}, \boldsymbol{w}_{i}^{\prime}\right)$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    14.&emsp;&emsp;&emsp;&emsp;&emsp;使用MBGD，根据最小化损失函数来更新价值网络（评论家网络）参数$\boldsymbol{w}$：
</font><br>
\begin{equation}
\nabla_{w} L(\boldsymbol{w}) \approx \frac{1}{N} \sum_{i}^{N}\left(y_{i}-Q\left(S_{i}, A_{i}, \boldsymbol{w}\right)\right) \nabla_{w} Q\left(S_{i}, A_{i}, \boldsymbol{w}\right)
\end{equation}
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    15.&emsp;&emsp;&emsp;&emsp;&emsp;<b>if</b> $t$ mod $d$ <b>then</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    16.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;使用MBGA法，根据最大化目标函数来更新策略网络（行动者网络）参数$\theta$： 
</font><br>
\begin{equation}
\left.\nabla_{\theta} \hat{J}_{\beta}(\boldsymbol{\theta}) \approx \frac{1}{N} \sum_{i} \nabla_{\theta} \mu\left(S_{i}, \boldsymbol{\theta}\right) \nabla_{a} Q\left(S_{i}, a, \boldsymbol{w}\right)\right|_{a=\mu\left(S_{i}, \boldsymbol{\theta}\right)} 
\end{equation}
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    17.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;软更新目标网络：$\left\{\begin{array}{l}\boldsymbol{w}^{\prime} \leftarrow \tau \boldsymbol{w}+(1-\tau) \boldsymbol{w}^{\prime} \\ \boldsymbol{\theta}^{\prime} \leftarrow \tau \boldsymbol{\theta}+(1-\tau) \boldsymbol{\theta}^{\prime}\end{array}\right.$
</font><br>
&emsp;&emsp;&emsp;&emsp;<font size=3.5>
    18.&emsp;&emsp;&emsp;<b>until</b> $t=\mathrm{T}-1$
</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>

In [1]:
import numpy as np
import torch
import gym
import os
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## USE CUDA

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Replay Buffer

In [3]:
class ReplayBuffer(object):
    def __init__(self, state_dim, action_dim, max_size=int(1e6)):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0

        self.state = np.zeros((max_size, state_dim))
        self.action = np.zeros((max_size, action_dim))
        self.next_state = np.zeros((max_size, state_dim))
        self.reward = np.zeros((max_size, 1))
        self.not_done = np.zeros((max_size, 1))

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def add(self, state, action, next_state, reward, done):
        self.state[self.ptr] = state
        self.action[self.ptr] = action
        self.next_state[self.ptr] = next_state
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)

        return (
            torch.FloatTensor(self.state[ind]).to(self.device),
            torch.FloatTensor(self.action[ind]).to(self.device),
            torch.FloatTensor(self.next_state[ind]).to(self.device),
            torch.FloatTensor(self.reward[ind]).to(self.device),
            torch.FloatTensor(self.not_done[ind]).to(self.device)
        )

In [4]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 256)
        self.l2 = nn.Linear(256, 256)
        self.l3 = nn.Linear(256, action_dim)

        self.max_action = max_action


    def forward(self, state):
        a = F.relu(self.l1(state))
        a = F.relu(self.l2(a))
        return self.max_action * torch.tanh(self.l3(a))


class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(state_dim + action_dim, 256)
        self.l2 = nn.Linear(256, 256)
        self.l3 = nn.Linear(256, 1)

        # Q2 architecture
        self.l4 = nn.Linear(state_dim + action_dim, 256)
        self.l5 = nn.Linear(256, 256)
        self.l6 = nn.Linear(256, 1)


    def forward(self, state, action):
        sa = torch.cat([state, action], 1)

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2


    def Q1(self, state, action):
        sa = torch.cat([state, action], 1)

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [9]:
actor1=Actor(17,6,1.0)
for ch in actor1.children():
    print(ch)
print("*********************")
critic1=Critic(17,6)
for ch in critic1.children():
    print(ch)

Linear(in_features=17, out_features=256, bias=True)
Linear(in_features=256, out_features=256, bias=True)
Linear(in_features=256, out_features=6, bias=True)
*********************
Linear(in_features=23, out_features=256, bias=True)
Linear(in_features=256, out_features=256, bias=True)
Linear(in_features=256, out_features=1, bias=True)
Linear(in_features=23, out_features=256, bias=True)
Linear(in_features=256, out_features=256, bias=True)
Linear(in_features=256, out_features=1, bias=True)


In [5]:
class TD3(object):
    def __init__(
        self,
        state_dim,
        action_dim,
        max_action,
        discount=0.99,
        tau=0.005,
        policy_noise=0.2,
        noise_clip=0.5,
        policy_freq=2
    ):

        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = copy.deepcopy(self.actor)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=3e-4)

        self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = copy.deepcopy(self.critic)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=3e-4)

        self.max_action = max_action
        self.discount = discount
        self.tau = tau
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip
        self.policy_freq = policy_freq

        self.total_it = 0


    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()


    def train(self, replay_buffer, batch_size=100):
        self.total_it += 1

        # Sample replay buffer 
        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip)

            next_action = (
                self.actor_target(next_state) + noise
            ).clamp(-self.max_action, self.max_action)

            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + not_done * self.discount * target_Q

        # Get current Q estimates
        current_Q1, current_Q2 = self.critic(state, action)

        # Compute critic loss
        critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:

            # Compute actor losse
            actor_loss = -self.critic.Q1(state, self.actor(state)).mean()

            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Update the frozen target models
            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


    def save(self, filename):
        torch.save(self.critic.state_dict(), filename + "_critic")
        torch.save(self.critic_optimizer.state_dict(), filename + "_critic_optimizer")

        torch.save(self.actor.state_dict(), filename + "_actor")
        torch.save(self.actor_optimizer.state_dict(), filename + "_actor_optimizer")


    def load(self, filename):
        self.critic.load_state_dict(torch.load(filename + "_critic"))
        self.critic_optimizer.load_state_dict(torch.load(filename + "_critic_optimizer"))
        self.critic_target = copy.deepcopy(self.critic)

        self.actor.load_state_dict(torch.load(filename + "_actor"))
        self.actor_optimizer.load_state_dict(torch.load(filename + "_actor_optimizer"))
        self.actor_target = copy.deepcopy(self.actor)

In [6]:
# Runs policy for X episodes and returns average reward
# A fixed seed is used for the eval environment
def eval_policy(policy, env_name, seed, eval_episodes=10):
    eval_env = gym.make(env_name)
    eval_env.seed(seed + 100)

    avg_reward = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        while not done:
            action = policy.select_action(np.array(state))
            state, reward, done, _ = eval_env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f}")
    print("---------------------------------------")
    return avg_reward

In [7]:
policy="TD3"
env_name="Walker2d-v2"          # OpenAI gym environment name
seed=0                        # Sets Gym, PyTorch and Numpy seeds
start_timesteps=25e3         # Time steps initial random policy is used
eval_freq=5e3               # How often (time steps) we evaluate
max_timesteps=1e6   # Max time steps to run environment
expl_noise=0.1                 # Std of Gaussian exploration noise
batch_size=256      # Batch size for both actor and critic
discount=0.99                 # Discount factor
tau=0.005                     # Target network update rate
policy_noise=0.2              # Noise added to target policy during critic update
noise_clip=0.5                # Range to clip target policy noise
policy_freq=2                 # Frequency of delayed policy updates
save_model="store_true"       # Save model and optimizer parameters
load_model=""                # Model load file name, "" doesn't load, "default" uses file_name

file_name = f"{policy}_{env_name}_{seed}"
print("---------------------------------------")
print(f"Policy: {policy}, Env: {env_name}, Seed: {seed}")
print("---------------------------------------")

if not os.path.exists("./results"):
    os.makedirs("./results")

if save_model and not os.path.exists("./models"):
    os.makedirs("./models")

env = gym.make(env_name)

# Set seeds
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
max_action = float(env.action_space.high[0])

kwargs = {
    "state_dim": state_dim,
    "action_dim": action_dim,
    "max_action": max_action,
    "discount": discount,
    "tau": tau,
    "policy_noise": policy_noise * max_action,
    "noise_clip": noise_clip * max_action,
    "policy_freq": policy_freq
}


policy = TD3(**kwargs)

if load_model != "":
    policy_file = file_name if load_model == "default" else load_model
    policy.load(f"./models/{policy_file}")

replay_buffer = ReplayBuffer(state_dim, action_dim)

# Evaluate untrained policy
evaluations = [eval_policy(policy, env_name, seed)]

state, done = env.reset(), False
episode_reward = 0
episode_timesteps = 0
episode_num = 0

for t in range(int(max_timesteps)):

    episode_timesteps += 1

    # Select action randomly or according to policy
    if t < start_timesteps:
        action = env.action_space.sample()
    else:
        action = (
            policy.select_action(np.array(state))
            + np.random.normal(0, max_action * expl_noise, size=action_dim)
        ).clip(-max_action, max_action)

    # Perform action
    next_state, reward, done, _ = env.step(action) 
    done_bool = float(done) if episode_timesteps < env._max_episode_steps else 0

    # Store data in replay buffer
    replay_buffer.add(state, action, next_state, reward, done_bool)

    state = next_state
    episode_reward += reward

    # Train agent after collecting sufficient data
    if t >= start_timesteps:
        policy.train(replay_buffer, batch_size)

    if done: 
        # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
        print(f"Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
        # Reset environment
        state, done = env.reset(), False
        episode_reward = 0
        episode_timesteps = 0
        episode_num += 1 

    # Evaluate episode
    if (t + 1) % eval_freq == 0:
        evaluations.append(eval_policy(policy, env_name, seed))
        np.save(f"./results/{file_name}", evaluations)
    
    if save_model: 
        policy.save(f"./models/{file_name}")

---------------------------------------
Policy: TD3, Env: Walker2d-v2, Seed: 0
---------------------------------------
---------------------------------------
Evaluation over 10 episodes: 7.883
---------------------------------------
Total T: 18 Episode Num: 1 Episode T: 18 Reward: 0.863
Total T: 33 Episode Num: 2 Episode T: 15 Reward: 0.775
Total T: 48 Episode Num: 3 Episode T: 15 Reward: -8.107
Total T: 68 Episode Num: 4 Episode T: 20 Reward: 2.861
Total T: 81 Episode Num: 5 Episode T: 13 Reward: -1.973
Total T: 98 Episode Num: 6 Episode T: 17 Reward: 1.578
Total T: 109 Episode Num: 7 Episode T: 11 Reward: -2.600
Total T: 122 Episode Num: 8 Episode T: 13 Reward: 2.055
Total T: 155 Episode Num: 9 Episode T: 33 Reward: 11.694
Total T: 184 Episode Num: 10 Episode T: 29 Reward: 7.228
Total T: 196 Episode Num: 11 Episode T: 12 Reward: -3.649
Total T: 216 Episode Num: 12 Episode T: 20 Reward: 2.729
Total T: 238 Episode Num: 13 Episode T: 22 Reward: -1.251
Total T: 269 Episode Num: 14 Episo

Total T: 2864 Episode Num: 140 Episode T: 14 Reward: -3.958
Total T: 2878 Episode Num: 141 Episode T: 14 Reward: 1.810
Total T: 2893 Episode Num: 142 Episode T: 15 Reward: -0.519
Total T: 2917 Episode Num: 143 Episode T: 24 Reward: 4.097
Total T: 2946 Episode Num: 144 Episode T: 29 Reward: 14.824
Total T: 2960 Episode Num: 145 Episode T: 14 Reward: 4.699
Total T: 2989 Episode Num: 146 Episode T: 29 Reward: 2.584
Total T: 3027 Episode Num: 147 Episode T: 38 Reward: 1.286
Total T: 3045 Episode Num: 148 Episode T: 18 Reward: 1.103
Total T: 3101 Episode Num: 149 Episode T: 56 Reward: 32.155
Total T: 3109 Episode Num: 150 Episode T: 8 Reward: -2.982
Total T: 3129 Episode Num: 151 Episode T: 20 Reward: -1.912
Total T: 3157 Episode Num: 152 Episode T: 28 Reward: 8.522
Total T: 3172 Episode Num: 153 Episode T: 15 Reward: -3.467
Total T: 3194 Episode Num: 154 Episode T: 22 Reward: -0.799
Total T: 3223 Episode Num: 155 Episode T: 29 Reward: 9.442
Total T: 3242 Episode Num: 156 Episode T: 19 Rewa

Total T: 5627 Episode Num: 278 Episode T: 25 Reward: 10.907
Total T: 5640 Episode Num: 279 Episode T: 13 Reward: -1.665
Total T: 5665 Episode Num: 280 Episode T: 25 Reward: 8.104
Total T: 5675 Episode Num: 281 Episode T: 10 Reward: -0.196
Total T: 5686 Episode Num: 282 Episode T: 11 Reward: -2.271
Total T: 5719 Episode Num: 283 Episode T: 33 Reward: 8.561
Total T: 5728 Episode Num: 284 Episode T: 9 Reward: -2.376
Total T: 5759 Episode Num: 285 Episode T: 31 Reward: -1.148
Total T: 5779 Episode Num: 286 Episode T: 20 Reward: -6.399
Total T: 5809 Episode Num: 287 Episode T: 30 Reward: -3.334
Total T: 5831 Episode Num: 288 Episode T: 22 Reward: 0.140
Total T: 5851 Episode Num: 289 Episode T: 20 Reward: -0.119
Total T: 5863 Episode Num: 290 Episode T: 12 Reward: 0.794
Total T: 5879 Episode Num: 291 Episode T: 16 Reward: -0.785
Total T: 5894 Episode Num: 292 Episode T: 15 Reward: -0.445
Total T: 5910 Episode Num: 293 Episode T: 16 Reward: -2.775
Total T: 5927 Episode Num: 294 Episode T: 17 

Total T: 8560 Episode Num: 419 Episode T: 11 Reward: -3.072
Total T: 8595 Episode Num: 420 Episode T: 35 Reward: 2.889
Total T: 8612 Episode Num: 421 Episode T: 17 Reward: -0.215
Total T: 8639 Episode Num: 422 Episode T: 27 Reward: -2.735
Total T: 8664 Episode Num: 423 Episode T: 25 Reward: 9.905
Total T: 8685 Episode Num: 424 Episode T: 21 Reward: 9.672
Total T: 8698 Episode Num: 425 Episode T: 13 Reward: -0.227
Total T: 8711 Episode Num: 426 Episode T: 13 Reward: 0.171
Total T: 8734 Episode Num: 427 Episode T: 23 Reward: -4.664
Total T: 8777 Episode Num: 428 Episode T: 43 Reward: 10.907
Total T: 8792 Episode Num: 429 Episode T: 15 Reward: -1.877
Total T: 8805 Episode Num: 430 Episode T: 13 Reward: 1.071
Total T: 8844 Episode Num: 431 Episode T: 39 Reward: 5.077
Total T: 8859 Episode Num: 432 Episode T: 15 Reward: 1.975
Total T: 8869 Episode Num: 433 Episode T: 10 Reward: -0.908
Total T: 8882 Episode Num: 434 Episode T: 13 Reward: 0.218
Total T: 8897 Episode Num: 435 Episode T: 15 Rew

Total T: 11266 Episode Num: 555 Episode T: 33 Reward: 11.064
Total T: 11285 Episode Num: 556 Episode T: 19 Reward: -0.856
Total T: 11302 Episode Num: 557 Episode T: 17 Reward: 4.289
Total T: 11317 Episode Num: 558 Episode T: 15 Reward: -2.102
Total T: 11341 Episode Num: 559 Episode T: 24 Reward: -1.828
Total T: 11361 Episode Num: 560 Episode T: 20 Reward: 1.399
Total T: 11376 Episode Num: 561 Episode T: 15 Reward: 3.854
Total T: 11391 Episode Num: 562 Episode T: 15 Reward: 1.496
Total T: 11404 Episode Num: 563 Episode T: 13 Reward: -4.943
Total T: 11421 Episode Num: 564 Episode T: 17 Reward: 5.138
Total T: 11441 Episode Num: 565 Episode T: 20 Reward: -5.927
Total T: 11453 Episode Num: 566 Episode T: 12 Reward: -0.165
Total T: 11478 Episode Num: 567 Episode T: 25 Reward: 14.339
Total T: 11494 Episode Num: 568 Episode T: 16 Reward: 4.875
Total T: 11519 Episode Num: 569 Episode T: 25 Reward: 11.570
Total T: 11550 Episode Num: 570 Episode T: 31 Reward: 14.996
Total T: 11565 Episode Num: 57

...

Total T: 638318 Episode Num: 3039 Episode T: 1000 Reward: 4249.230
Total T: 639318 Episode Num: 3040 Episode T: 1000 Reward: 4267.123
---------------------------------------
Evaluation over 10 episodes: 4172.185
---------------------------------------
Total T: 640318 Episode Num: 3041 Episode T: 1000 Reward: 4286.727
Total T: 641318 Episode Num: 3042 Episode T: 1000 Reward: 4200.828
Total T: 642228 Episode Num: 3043 Episode T: 910 Reward: 3987.868
Total T: 643085 Episode Num: 3044 Episode T: 857 Reward: 3711.723
Total T: 643505 Episode Num: 3045 Episode T: 420 Reward: 1730.029
Total T: 644040 Episode Num: 3046 Episode T: 535 Reward: 2213.184
---------------------------------------
Evaluation over 10 episodes: 3615.827
---------------------------------------
Total T: 645040 Episode Num: 3047 Episode T: 1000 Reward: 4248.520
Total T: 646040 Episode Num: 3048 Episode T: 1000 Reward: 4070.327
Total T: 647040 Episode Num: 3049 Episode T: 1000 Reward: 4188.960
Total T: 648040 Episode Num: 30

Total T: 722340 Episode Num: 3132 Episode T: 1000 Reward: 4123.218
Total T: 723340 Episode Num: 3133 Episode T: 1000 Reward: 4161.516
Total T: 724340 Episode Num: 3134 Episode T: 1000 Reward: 4043.134
---------------------------------------
Evaluation over 10 episodes: 3892.184
---------------------------------------
Total T: 725340 Episode Num: 3135 Episode T: 1000 Reward: 4199.475
Total T: 726340 Episode Num: 3136 Episode T: 1000 Reward: 4202.914
Total T: 727340 Episode Num: 3137 Episode T: 1000 Reward: 4315.049
Total T: 728340 Episode Num: 3138 Episode T: 1000 Reward: 4210.778
Total T: 729340 Episode Num: 3139 Episode T: 1000 Reward: 4186.181
---------------------------------------
Evaluation over 10 episodes: 4077.844
---------------------------------------
Total T: 730340 Episode Num: 3140 Episode T: 1000 Reward: 4075.218
Total T: 731171 Episode Num: 3141 Episode T: 831 Reward: 3463.781
Total T: 732171 Episode Num: 3142 Episode T: 1000 Reward: 4233.412
Total T: 733171 Episode Num:

Total T: 810623 Episode Num: 3223 Episode T: 1000 Reward: 4410.662
Total T: 811623 Episode Num: 3224 Episode T: 1000 Reward: 4472.352
Total T: 812623 Episode Num: 3225 Episode T: 1000 Reward: 4354.994
Total T: 813623 Episode Num: 3226 Episode T: 1000 Reward: 4309.307
Total T: 814623 Episode Num: 3227 Episode T: 1000 Reward: 4220.009
---------------------------------------
Evaluation over 10 episodes: 4180.755
---------------------------------------
Total T: 815623 Episode Num: 3228 Episode T: 1000 Reward: 4372.823
Total T: 816623 Episode Num: 3229 Episode T: 1000 Reward: 4343.525
Total T: 817623 Episode Num: 3230 Episode T: 1000 Reward: 4280.633
Total T: 818623 Episode Num: 3231 Episode T: 1000 Reward: 4549.305
Total T: 819623 Episode Num: 3232 Episode T: 1000 Reward: 4476.145
Total T: 819903 Episode Num: 3233 Episode T: 280 Reward: 934.012
---------------------------------------
Evaluation over 10 episodes: 4531.719
---------------------------------------
Total T: 820903 Episode Num: 

Total T: 899741 Episode Num: 3316 Episode T: 1000 Reward: 4353.110
---------------------------------------
Evaluation over 10 episodes: 4380.195
---------------------------------------
Total T: 900741 Episode Num: 3317 Episode T: 1000 Reward: 4312.863
Total T: 901741 Episode Num: 3318 Episode T: 1000 Reward: 4251.099
Total T: 902741 Episode Num: 3319 Episode T: 1000 Reward: 4346.339
Total T: 903741 Episode Num: 3320 Episode T: 1000 Reward: 4409.965
Total T: 904741 Episode Num: 3321 Episode T: 1000 Reward: 4381.219
---------------------------------------
Evaluation over 10 episodes: 4409.121
---------------------------------------
Total T: 905741 Episode Num: 3322 Episode T: 1000 Reward: 4434.765
Total T: 906741 Episode Num: 3323 Episode T: 1000 Reward: 4397.271
Total T: 907741 Episode Num: 3324 Episode T: 1000 Reward: 4362.079
Total T: 908741 Episode Num: 3325 Episode T: 1000 Reward: 4412.058
Total T: 909741 Episode Num: 3326 Episode T: 1000 Reward: 4214.954
---------------------------

---------------------------------------
Evaluation over 10 episodes: 4513.698
---------------------------------------
Total T: 990005 Episode Num: 3407 Episode T: 1000 Reward: 4483.452
Total T: 991005 Episode Num: 3408 Episode T: 1000 Reward: 4483.843
Total T: 992005 Episode Num: 3409 Episode T: 1000 Reward: 4450.741
Total T: 993005 Episode Num: 3410 Episode T: 1000 Reward: 4502.575
Total T: 994005 Episode Num: 3411 Episode T: 1000 Reward: 4420.689
---------------------------------------
Evaluation over 10 episodes: 4543.741
---------------------------------------
Total T: 995005 Episode Num: 3412 Episode T: 1000 Reward: 4496.086
Total T: 996005 Episode Num: 3413 Episode T: 1000 Reward: 4513.234
Total T: 997005 Episode Num: 3414 Episode T: 1000 Reward: 4541.839
Total T: 998005 Episode Num: 3415 Episode T: 1000 Reward: 4609.012
Total T: 999005 Episode Num: 3416 Episode T: 1000 Reward: 4557.577
---------------------------------------
Evaluation over 10 episodes: 4551.109
----------------

In [10]:
state_dim

17